In [2]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('../datasets/T20-Edit - Sheet1.csv')

In [3]:
unique_match_ids = dataset.match_id.unique()

for i in unique_match_ids:
    unique_bowlers = dataset[(dataset['match_id'] == i)]['bowler'].unique()
    for j in unique_bowlers:
        wickets = len(dataset.loc[(dataset['match_id']==i) & (dataset['bowler'] == j) & (dataset['wicket_type'] != '.')])
        dataset.loc[(dataset['match_id']==i) & (dataset['bowler'] == j), 'wickets_individual'] = wickets

In [4]:
for i in unique_match_ids:
    unique_bowlers = dataset[(dataset['match_id'] == i)]['bowler'].unique()
    for j in unique_bowlers:
        wick = dataset.loc[(dataset['match_id']==i) & (dataset['bowler'] == j)]['wickets_individual'].unique()
        if wick[0] <= 1:
            dataset.loc[(dataset['match_id']==i) & (dataset['bowler'] == j), 'rating'] = 1
        elif (wick[0] >= 2) and (wick[0] <= 3):
            dataset.loc[(dataset['match_id']==i) & (dataset['bowler'] == j), 'rating'] = 2
        elif (wick[0] >= 4):
            dataset.loc[(dataset['match_id']==i) & (dataset['bowler'] == j), 'rating'] = 3

In [5]:
dataset.head()

,match_id,season,start_date,venue,pitch_conditon,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wicket_type,player_dismissed,wickets_individual,rating
0,1162727,2018/19,22-10-2018,Tolerance Oval,bowling,1,0.1,United Arab Emirates,Australia,Ashfaq Ahmed,Chirag Suri,NM Coulter-Nile,0,0,.,.,2.0,2.0
1,1162727,2018/19,22-10-2018,Tolerance Oval,bowling,1,0.2,United Arab Emirates,Australia,Ashfaq Ahmed,Chirag Suri,NM Coulter-Nile,0,0,.,.,2.0,2.0
2,1162727,2018/19,22-10-2018,Tolerance Oval,bowling,1,0.3,United Arab Emirates,Australia,Ashfaq Ahmed,Chirag Suri,NM Coulter-Nile,0,0,caught,Ashfaq Ahmed,2.0,2.0
3,1162727,2018/19,22-10-2018,Tolerance Oval,bowling,1,0.4,United Arab Emirates,Australia,Chirag Suri,Rohan Mustafa,NM Coulter-Nile,0,0,.,.,2.0,2.0
4,1162727,2018/19,22-10-2018,Tolerance Oval,bowling,1,0.5,United Arab Emirates,Australia,Chirag Suri,Rohan Mustafa,NM Coulter-Nile,0,0,.,.,2.0,2.0


In [6]:
dataset = dataset.drop(['match_id', 'season', 'start_date', 'innings', 'ball', 'non_striker','striker','runs_off_bat','extras', 'wicket_type','player_dismissed','wickets_individual'],axis=1)

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [8]:
le_wick_venue = LabelEncoder()
le_wick_pitchcond = LabelEncoder()
le_wick_bat = LabelEncoder()
le_wick_bowl = LabelEncoder()
le_wick_bowler = LabelEncoder()

In [9]:
dataset['venue'] = le_wick_venue.fit_transform(dataset['venue'])
dataset['pitch_conditon'] = le_wick_pitchcond.fit_transform(dataset['pitch_conditon'])
dataset['batting_team'] = le_wick_bat.fit_transform(dataset['batting_team'])
dataset['bowling_team'] = le_wick_bowl.fit_transform(dataset['bowling_team'])
dataset['bowler'] = le_wick_bowler.fit_transform(dataset['bowler'])

In [11]:
X = dataset.drop(['rating'],axis=1)
Y = dataset['rating']

In [12]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y, test_size = 0.3)

In [13]:
# from sklearn.linear_model import LinearRegression

# linear_reg = LinearRegression()
# linear_reg.fit(Xtrain, Ytrain)

# linear_reg_predict = linear_reg.predict(Xtest)
# print(linear_reg.score(Xtest,Ytest))

In [14]:
from sklearn.naive_bayes import GaussianNB

naive_bayes_algo = GaussianNB()

naive_bayes_algo.fit(Xtrain,Ytrain)

y_pred_nb = naive_bayes_algo.predict(Xtest)

print(accuracy_score(Ytest,y_pred_nb))

0.6518260948614805


In [15]:
from sklearn.ensemble import RandomForestClassifier

random_forest_class  = RandomForestClassifier()

random_forest_class.fit(Xtrain,Ytrain)

y_pred_rfc = random_forest_class.predict(Xtest)

print(accuracy_score(Ytest,y_pred_rfc))

0.9006613486057135


In [17]:
from sklearn.tree import DecisionTreeClassifier

dec_tree_class = DecisionTreeClassifier(criterion = 'entropy')

dec_tree_class.fit(Xtrain,Ytrain)

y_pred_dtc = dec_tree_class.predict(Xtest)

print(accuracy_score(Ytest,y_pred_dtc))

0.8999568191632008


In [18]:
import pickle

bowler_wickets = {'model': random_forest_class, 'le_wick_venue':le_wick_venue, 'le_wick_pitchcond':le_wick_pitchcond,
'le_wick_bat':le_wick_bat, 'le_wick_bowl':le_wick_bowl, 'le_wick_bowler':le_wick_bowler}

with open('bowler_wickets.pkl', 'wb') as file:
    pickle.dump(bowler_wickets, file)